In [1]:
import findspark
import os
findspark.init('/usr/lib/apache_spark')

import pyspark
sc = pyspark.SparkContext()

In [31]:
from pathlib import Path
from zipfile import ZipFile
import requests
from math import ceil

taxidir = Path('taxidata')
if not taxidir.exists(): taxidir.mkdir(parents=True)

In [34]:
zipped = ZipFile(taxidir / 'trip_data_4.csv.zip')

AttributeError: 'PosixPath' object has no attribute 'seek'

In [53]:
from IPython.display import display
from ipywidgets import FloatProgress, HTML
import time
progress = FloatProgress(max=980, description='fname')
display(progress)

for i in range(980):
    time.sleep(0.001)
    progress.value = i
    progress.max
    progress.description = 'fname: %4d / %4d' % (i, 980)

In [58]:

txt.value = txt.value + 'test str.\n'
txt.value
display(txt)

In [60]:
txt = HTML()
display(txt)

for n in range(1, 13):
    trip_url = 'https://nyctaxitrips.blob.core.windows.net/data/trip_data_%d.csv.zip' % n
    path = taxidir / ('trip_data_%d.csv.zip' % n)
    if path.exists():
        try:
            unzipped = ZipFile(str(path))
            continue
        except:
            txt.value = txt.value + ('Path %s found, but zipfile corrupt. Removing.\n' % path)
            path.unlink()
    
    request = requests.get(trip_url, stream=True)
    length = int(request.headers.get('Content-Length', 0))
    
    chunk_size = 1024*1024
    num_chunks = ceil(length / chunk_size)
    
    progress = FloatProgress(max=num_chunks, description=str(path))
    display(progress)
    
    with path.open('wb') as fd:
        for chunk_num, chunk in enumerate(request.iter_content(chunk_size), 1):
            fd.write(chunk)
            progress.value = chunk_num
            progress.description = '%4d / %d  %s' % (chunk_num, num_chunks, str(path))

KeyboardInterrupt: 